# Big Data Mining, 1st Assignment solution
## Vangelis Christou, p2821805

### Import libraries

In [226]:
#read
import pandas as pd 
df = pd.read_csv("groceries.csv",sep=";") 

#preview data
df.head(20)




,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
0,1,75,male,married,primary,20000,very_good,3,retired,"citrus fruit,semi-finished bread,margarine,rea..."
1,2,61,female,single,secondary,28000,good,1,housemaid,"tropical fruit,yogurt,coffee"
2,3,32,male,single,secondary,34000,very_good,1,blue-collar,whole milk
3,4,62,male,married,primary,31000,very_good,3,blue-collar,"pip fruit,yogurt,cream cheese,meat spreads"
4,5,66,female,married,secondary,19000,good,3,retired,"other vegetables,whole milk,condensed milk,lon..."
5,6,55,female,single,secondary,35000,very_good,1,unemployed,"whole milk,butter,yogurt,rice,abrasive cleaner"
6,7,23,female,married,tertiary,21000,good,3,housemaid,rolls/buns
7,8,26,female,single,secondary,30000,good,2,blue-collar,"other vegetables,UHT-milk,rolls/buns,bottled b..."
8,9,29,female,married,secondary,32000,very_good,3,blue-collar,potted plants
9,10,57,female,married,secondary,26000,good,3,entrepreneur,"whole milk,cereals"


In [227]:
#view default  data types
df.dtypes

Customer_ID              int64
Age                     object
Sex                     object
Marital_Status          object
Education               object
Income                  object
Customer_Rating         object
Persons_in_Household     int64
Occupation              object
Groceries               object
dtype: object

In [228]:
#replace ' ' values with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

#set proper data types
#numeric
df["Age"] = pd.to_numeric(df["Age"],downcast='integer')
df["Income"] = pd.to_numeric(df["Income"],downcast='integer')
df["Persons_in_Household"] = pd.to_numeric(df["Persons_in_Household"])

#nominal
df["Sex"] = df["Sex"].astype('category')
df["Marital_Status"] = df["Marital_Status"].astype('category')
df["Occupation"] = df["Occupation"].astype('category')

#ordinal
cRatings = pd.Categorical(df["Customer_Rating"], categories=["poor", "fair", "good", "very_good", "excellent"], ordered=True)
df.Customer_Rating = cRatings
cEducation = pd.Categorical(df["Education"], categories=["primary", "secondary", "tertiary"], ordered=True)
df.Education = cEducation



#fill Nan values with mean
df.fillna(df.mean().astype(int), inplace=True)
df["Age"] = df["Age"].astype(np.int64) 
df["Income"] = df["Income"].astype(np.int64) 

# convert Groceries to set of values
df.Groceries= df.Groceries.str.split(",") 

df.dtypes





Customer_ID                int64
Age                        int64
Sex                     category
Marital_Status          category
Education               category
Income                     int64
Customer_Rating         category
Persons_in_Household       int64
Occupation              category
Groceries                 object
dtype: object

In [229]:

df.head(20)

,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
0,1,75,male,married,primary,20000,very_good,3,retired,"[citrus fruit, semi-finished bread, margarine,..."
1,2,61,female,single,secondary,28000,good,1,housemaid,"[tropical fruit, yogurt, coffee]"
2,3,32,male,single,secondary,34000,very_good,1,blue-collar,[whole milk]
3,4,62,male,married,primary,31000,very_good,3,blue-collar,"[pip fruit, yogurt, cream cheese, meat spreads]"
4,5,66,female,married,secondary,19000,good,3,retired,"[other vegetables, whole milk, condensed milk,..."
5,6,55,female,single,secondary,35000,very_good,1,unemployed,"[whole milk, butter, yogurt, rice, abrasive cl..."
6,7,23,female,married,tertiary,21000,good,3,housemaid,[rolls/buns]
7,8,26,female,single,secondary,30000,good,2,blue-collar,"[other vegetables, UHT-milk, rolls/buns, bottl..."
8,9,29,female,married,secondary,32000,very_good,3,blue-collar,[potted plants]
9,10,57,female,married,secondary,26000,good,3,entrepreneur,"[whole milk, cereals]"


In [230]:
import itertools
import time
def jaccard_sim(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return round(float(intersection) / union, 2)

def pairwiseJaccard(cust):
    sim={}
    print("Computing pair wise similarities")
    start_time = time.time()
    for pair in itertools.combinations(cust, r=2):
        s=jaccard_sim(cust[pair[0]],cust[pair[1]])
        sim[tuple([pair[0],pair[1]])]=s
    
    end_time = time.time()
    print("Success after: " ,end_time-start_time)
    return sim



In [231]:
cust_dict = dict(zip(df.Customer_ID, df.Groceries))

short_c = dict(list(cust_dict.items())[0:1000])



In [232]:
sim = pairwiseJaccard(cust_dict)

Computing pair wise similarities
Success after:  77.34021639823914


In [235]:
import operator

def GetSimilarCustomers(customerID,head, sim):
    filtered_dict = {k:v for (k,v) in sim.items() if k[0] == customerID}
    sorted_d = dict(sorted(filtered_dict.items(), key=operator.itemgetter(1),reverse=True))
    return dict(list(sorted_d.items())[0:head])


In [242]:
customerids = [73, 563, 1603, 2200, 3703, 4263, 5300, 6129, 7800, 8555]
for i in customerids: 
    print(GetSimilarCustomers(i,10,sim))
    print("----\n")


{(73, 7393): 0.4444444444444444, (73, 8719): 0.4, (73, 3888): 0.375, (73, 4468): 0.375, (73, 7827): 0.375, (73, 9328): 0.375, (73, 8704): 0.35714285714285715, (73, 2300): 0.3333333333333333, (73, 3282): 0.3333333333333333, (73, 4494): 0.3333333333333333}
----

{(563, 652): 1.0, (563, 768): 1.0, (563, 1010): 1.0, (563, 1236): 1.0, (563, 1562): 1.0, (563, 1629): 1.0, (563, 1652): 1.0, (563, 1787): 1.0, (563, 1819): 1.0, (563, 1830): 1.0}
----

{(1603, 8345): 1.0, (1603, 1767): 0.6666666666666666, (1603, 2337): 0.6666666666666666, (1603, 5119): 0.6666666666666666, (1603, 6328): 0.6666666666666666, (1603, 6379): 0.6666666666666666, (1603, 7559): 0.6666666666666666, (1603, 9705): 0.6666666666666666, (1603, 3154): 0.6, (1603, 1803): 0.5}
----

{(2200, 2414): 0.5, (2200, 2421): 0.5, (2200, 2626): 0.5, (2200, 2650): 0.5, (2200, 2694): 0.5, (2200, 2806): 0.5, (2200, 2874): 0.5, (2200, 3194): 0.5, (2200, 3434): 0.5, (2200, 3570): 0.5}
----

{(3703, 8871): 0.375, (3703, 6311): 0.3333333333333333,